<a href="https://colab.research.google.com/github/PongsatornWaiyaworn/TumorMRITrainModelAI/blob/main/Brain_Tumor_MRI_Train_Model_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install ultralytics -qq
!pip install roboflow -qq
!pip install pydicom -qq
!pip install opencv-python -qq
!pip install numpy -qq
!pip install albumentations -qq

In [9]:
import pydicom
import cv2
import os
import numpy as np
from ultralytics import YOLO
from roboflow import Roboflow
from google.colab import drive
import albumentations as A

In [10]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
def read_dicom(dicom_path):
    dicom_data = pydicom.dcmread(dicom_path)
    image = dicom_data.pixel_array

    # ตรวจสอบว่าเป็นภาพ MONOCHROME1 หรือไม่
    if dicom_data.PhotometricInterpretation == "MONOCHROME1":
        image = np.max(image) - image

    # Normalize ให้เป็นช่วง 0-255
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
    image = np.uint8(image)

    return image

In [12]:
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussianBlur(p=0.2)
])

In [13]:
def dicom_to_png(dicom_path, output_path):
    image = read_dicom(dicom_path)
    cv2.imwrite(output_path, image)

In [14]:
# ฟังก์ชันปรับปรุง Contrast ด้วย CLAHE
def apply_clahe(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    return cv2.merge([enhanced, enhanced, enhanced])  # เปลี่ยนเป็น BGRdef apply_clahe(image):

In [15]:
def resize_image(image):
    return cv2.resize(image, (640, 640))

In [16]:
def save_yolo_annotation(output_dir, filename, class_id, x_center, y_center, width, height):
    annotation = f"{class_id} {x_center} {y_center} {width} {height}"
    with open(f"{output_dir}/{filename}.txt", 'w') as file:
        file.write(annotation)

In [17]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [18]:
def process_images(input_dir, output_dir, model):
    for subfolder in os.listdir(input_dir):
        subfolder_path = os.path.join(input_dir, subfolder)
        if os.path.isdir(subfolder_path):
            output_subfolder_path = os.path.join(output_dir, subfolder)
            create_dir(output_subfolder_path)

            for file in os.listdir(subfolder_path):
                if file.endswith(('.jpg', '.png', '.jpeg', '.dcm')):
                    input_image_path = os.path.join(subfolder_path, file)

                    if file.endswith('.dcm'):
                        dicom_to_png(input_image_path, input_image_path.replace('.dcm', '.png'))
                        image = cv2.imread(input_image_path.replace('.dcm', '.png'))
                    else:
                        image = cv2.imread(input_image_path)

                    augmented_image = augment(image=image)["image"]
                    resized_image = resize_image(augmented_image)

                    enhanced_image = apply_clahe(resized_image)

                    results = model(input_image_path, save=False, task="detect")

                    for result in results:
                        for box in result.boxes:
                            x1, y1, x2, y2 = map(int, box.xyxy[0])
                            confidence = box.conf[0]
                            class_id = int(box.cls[0])

                            x_center = (x1 + x2) / 2 / enhanced_image.shape[1]
                            y_center = (y1 + y2) / 2 / enhanced_image.shape[0]
                            width = (x2 - x1) / enhanced_image.shape[1]
                            height = (y2 - y1) / enhanced_image.shape[0]

                            save_yolo_annotation(output_subfolder_path, file.split('.')[0], class_id, x_center, y_center, width, height)

                            label = f"{model.names[class_id]} {confidence:.2f}"
                            cv2.rectangle(enhanced_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                            cv2.putText(enhanced_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    output_image_path = os.path.join(output_subfolder_path, file)
                    cv2.imwrite(output_image_path, enhanced_image)

In [19]:
rf = Roboflow(api_key="LNYqYsggjofCU0MkR20b")
project = rf.workspace("clahe-8x8-dan-median").project("clahe-8x8-dan-median-2")
version = project.version(3)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [20]:
model_path = "/content/best.pt"
model = YOLO(model_path)

In [34]:
# !tar -xvf /content/train2vGliomaandNo.tar -C /content/trainMe
# !tar -xvf /content/BrainTumorMRIDataset.tar -C /content/dataset

In [35]:
import tarfile

file_path = '/content/BrainTumorMRIDataset.tar'
extract_path = '/content/dataset/'

with tarfile.open(file_path, 'r') as tar:
    tar.extractall(path=extract_path)

print("Extraction complete!")


Extraction complete!


In [23]:
train_results = model.train(
    data="/content/CLAHE-8x8-dan-median-2-3/data.yaml",
    epochs=1,
    imgsz=640,
    save=True
)

Ultralytics 8.3.73 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/best.pt, data=/content/CLAHE-8x8-dan-median-2-3/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

100%|██████████| 5.35M/5.35M [00:00<00:00, 93.9MB/s]


AMP: checks passed ✅


train: Scanning /content/trainMe/train2 vGliomaandNo... 2416 images, 1595 backgrounds, 0 corrupt: 100%|██████████| 2416/2416 [00:00<00:00, 3123.16it/s]


train: New cache created: /content/trainMe/train2 vGliomaandNo.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train4
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      2.45G      1.226      1.429      1.343         12        640: 100%|██████████| 151/151 [00:59<00:00,  2.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:14<00:00,  1.80it/s]


                   all        855        855      0.854      0.785      0.885      0.632

1 epochs completed in 0.023 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 5.5MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.73 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:13<00:00,  1.93it/s]


                   all        855        855      0.852      0.786      0.885      0.633
                Glioma        285        285      0.606      0.718      0.725      0.409
            Meningioma        284        284       0.99      0.768      0.964      0.807
       Pituitary tumor        286        286      0.962      0.874      0.965      0.682
Speed: 0.4ms preprocess, 2.8ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/train4


In [36]:
input_dir = "/content/dataset/Brain Tumor MRI Dataset/Testing"
output_dir = "/content/output"

In [ ]:
process_images(input_dir, output_dir, model)

print("Processing complete. Bounding box images saved at:", output_dir)


image 1/1 /content/dataset/Brain Tumor MRI Dataset/Testing/Meningioma/Te-me_0262.jpg: 640x640 1 Glioma, 21.0ms
Speed: 9.0ms preprocess, 21.0ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/dataset/Brain Tumor MRI Dataset/Testing/Meningioma/Te-me_0152.jpg: 640x640 1 Meningioma, 15.7ms
Speed: 4.2ms preprocess, 15.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/dataset/Brain Tumor MRI Dataset/Testing/Meningioma/Te-me_0165.jpg: 640x640 1 Glioma, 14.3ms
Speed: 6.3ms preprocess, 14.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/dataset/Brain Tumor MRI Dataset/Testing/Meningioma/Te-me_0092.jpg: 640x512 1 Glioma, 76.1ms
Speed: 4.4ms preprocess, 76.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /content/dataset/Brain Tumor MRI Dataset/Testing/Meningioma/Te-me_0149.jpg: 640x640 1 Meningioma, 19.1ms
Speed: 4.3ms preprocess, 19.1ms inference, 2